In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import os
import seaborn as sns
import statistics
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from utils.generateFinalRow import generateFinalRows
# from utils.generateRelativeRow import getRelativeRows


ratioKeys = ['psRatio', 'peRatio', 'priceToBook', 'evToEbitda', 'evToEbit', 'priceToFreeCashFlow', 'evToSales', 'evToGrossProfit', 'priceToGrossProfit']
yKeys = ['yAdjustedPriceSpinoffExcl1Year', 'yAdjustedPrice1Year', 'yPrice1Year']
y2Keys = ['yAdjustedPriceSpinoffExcl2Year', 'yAdjustedPrice2Year', 'yPrice2Year']
numericalKeys = ratioKeys + yKeys + y2Keys

resultsPath = '../data/results-no-alpha/'
# TODO: NOT SURE IF THIS IS DEPRECATED

In [ ]:
fileNames = os.listdir('../data/results-no-alpha')

allCompanies = []
for fileName in fileNames:
    with open('../data/results-no-alpha/' + fileName) as json_data:
        data = json.load(json_data)
        allCompanies.append(data)

spyData = {}

for year in range(2015, 2026):
    for month in range(1,13):
        currentAllData = pd.DataFrame()
        i = 0
        for company in allCompanies:
            yearData = company.get(year, 0)
            if yearData:
                monthData = yearData.get(month, 0)
                if monthData:
                    currCompany = pd.DataFrame({
                        'price': monthData['price'],
                        'adjustedTotalPrice': monthData['adjustedTotalPrice'],
                        'adjustedTotalPriceSpinoffExcl': monthData['adjustedTotalPriceSpinoffExcl']
                    }, index=i)
                    i += 1
                    currentAllData = pd.concat([currentAllData, currCompany])
        stats = {}
        for key in ['price', 'adjustedTotalPrice', 'adjustedTotalPriceSpinoffExcl']:
            stats[key] = {
                'mean': currentAllData.loc[:, key].mean(),
                'median': currentAllData.loc[:, key].median()
            }
        spyData[str(month) + '-' + str(year)] = stats


In [3]:
    # Generate y-no-alpha
    # results(no-alpha) -> y-no-alpha
    resultsPath = '../data/extra-data/results/'
    fileNames = os.listdir(resultsPath)
    companyDf = pd.DataFrame()
    offsets = [1,2]
    for fileName in fileNames:
        with open(resultsPath + fileName) as json_data:
            data = json.load(json_data)
            companyDf, i = generateFinalRows(data, ratioKeys, fileName, 0, companyDf, offsets)
            
            companyDf.to_csv('../data/extra-data/y-no-alpha/'+fileName.replace('.json', '.csv'), sep='\t')
        # print('Done ' + fileName)

# df = pd.read_csv('../data/temp.csv', sep='\t')

F:\programowanie\studia\magisterka\sec_scrapper2\finchat\utils\generateFinalRow.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, toAdd])
F:\programowanie\studia\magisterka\sec_scrapper2\finchat\utils\generateFinalRow.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, toAdd])
F:\programowanie\studia\magisterka\sec_scrapper2\finchat\utils\generateFinalRow.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is depr

In [8]:
# fileNames = os.listdir('../data/results')
# TEST!!!! TODO:
# Change offset to param!


In [ ]:
from utils.generateRelativeRow import getRelativeRows
# if not os.path.isfile('../data/temp-relative.csv'):
fileNames = os.listdir(resultsPath)
companyDf = pd.DataFrame()
i = 0
offsets = [1,2]
for fileName in fileNames: 
    with open(resultsPath + fileName) as json_data:
        data = json.load(json_data)
        companyDf, i = getRelativeRows(data, ratioKeys, fileName, i, companyDf, offsets)
        break
companyDf.to_csv('../data/temp-relative.csv', sep='\t')
    
# relativeDf = pd.read_csv('../data/temp-relative.csv', sep='\t')

In [ ]:
# TODO: DROP evToSales evToGrossProfit and calculate other columns again. More data will be available

In [ ]:
# TODO: normalize ratios. Could have better results. 
dfNorm = pd.DataFrame(normalize(df[ratioKeys]), columns=ratioKeys)
dfY = df[yKeys + y2Keys]
dfNorm = dfNorm.join(dfY)

In [ ]:
dfRelativeNorm = pd.DataFrame(normalize(relativeDf[ratioKeys]), columns=ratioKeys)
dfY = df[yKeys + y2Keys]
dfRelativeNorm = dfRelativeNorm.join(dfY)

In [ ]:
# dfNorm
# relativeDf
dfRelativeNorm

In [ ]:
sns.set_theme(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
matrix = relativeDf[numericalKeys].corr()
sns.heatmap(matrix, cmap="Greens", annot=True, fmt=".2f", ).set_title("Not normalized RELATIVE values")

In [ ]:
matrix = dfRelativeNorm[numericalKeys].corr()
sns.heatmap(matrix, cmap="Greens", annot=True, fmt=".2f", ).set_title("Normalized RELATIVE values")

In [ ]:
matrix = df[numericalKeys].corr()
sns.heatmap(matrix, cmap="Greens", annot=True, fmt=".2f", ).set_title("Not normalized absolute values")

In [ ]:
matrix = dfNorm.corr()
sns.heatmap(matrix, cmap="Greens", annot=True, fmt=".2f", ).set_title("Normalized absolute values")

In [ ]:

# for i, key in enumerate(keys):
#     med = statistics.median(df[key])
#     mean = statistics.mean(df[key])
#     # print("key: {}, median: {}, mean: {}".format(key, med, round(mean, 2)))
#     
#     # TODO: outliers from returns too
#     q_low = df[key].quantile(0.01)
#     q_hi  = df[key].quantile(0.99)
#     
#     df_filtered = df[(df[key] < q_hi) & (df[key] > q_low)]
    # sns.scatterplot(data=df_filtered, x=key, y='yAdjustedTotalPrice')
    # plt.show()
